# IPL Win Probabilty Predictor in 2nd Innings



In [ ]:
import pandas as pd
import numpy as np


In [ ]:
delivery=pd.read_csv('deliveries.csv')
match=pd.read_csv('matches.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
delivery.head()

In [ ]:
match.shape

In [ ]:
delivery.shape

In [ ]:
total_score_df=delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

In [ ]:
total_score_df.shape

In [ ]:
total_score_df.head()

In [ ]:
# extracted only first inning total of every match
total_score_df=total_score_df[total_score_df['inning']==1]

In [ ]:
total_score_df.head()

In [ ]:

match_df=match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')
            

In [ ]:
match_df.head()

In [ ]:
match_df['team1'].value_counts()

In [ ]:
# Handling team names that got changed and teams that are not playing now
teams=[
   'Royal Challengers Bengaluru',
    'Gujarat Titans',
    'Lucknow Super Giants',
    'Punjab Kings',
    'Delhi Capitals',
    'Sunrisers Hyderabad',
    'Rajasthan Royals',
    'Kolkata Knight Riders',
    'Mumbai Indians',
    'Chennai Super Kings'
]
match_df['team1']=match_df['team1'].str.replace('Royal Challengers Bangalore','Royal Challengers Bengaluru')
match_df['team2']=match_df['team2'].str.replace('Royal Challengers Bangalore','Royal Challengers Bengaluru')

match_df['team1']=match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace('Kings XI Punjab','Punjab Kings')
match_df['team2']=match_df['team2'].str.replace('Kings XI Punjab','Punjab Kings')

match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df['team1'].nunique()

In [ ]:
match_df['method'].isnull().mean()*100

In [ ]:
match_df.head()

In [ ]:
# only taking matches without D/L into consideration
match_df=match_df[match_df['method'].isnull()]

In [ ]:
match_df.head()

In [ ]:
match_df['method'].value_counts()

In [ ]:

match_df.columns


In [ ]:
# extracting important columns from match_df to merge with delivery
match_df=match_df[['id','city','winner','total_runs']] 

In [ ]:
match_df.head()

In [ ]:
#joining match_df with delivery
delivery_df=match_df.merge(delivery,left_on='id',right_on='match_id')


In [ ]:
# extracting 2nd inning data from delivery_df
delivery_df=delivery_df[delivery_df['inning']==2]


In [ ]:
 delivery_df.head(15)

In [ ]:
 delivery_df['current_score'] = (
    delivery_df.groupby('match_id')['total_runs_y']
    .cumsum()
)


In [ ]:
delivery_df['runs_left']=delivery_df['total_runs_x']-delivery_df['current_score']+1

In [ ]:
delivery_df.sample(5)

In [ ]:
delivery_df['balls_left']=120-(delivery_df['over']*6+delivery_df['ball'])

In [ ]:
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].notna().astype(int)
delivery_df['wickets'] = 10 - delivery_df.groupby('match_id')['player_dismissed'].cumsum()
delivery_df['wickets'] = delivery_df['wickets'].clip(lower=0)

In [ ]:
delivery_df.head()

In [ ]:
delivery_df['crr']=delivery_df['current_score']*6/(120-delivery_df['balls_left'])

In [ ]:
delivery_df['rrr']=delivery_df['runs_left']*6/delivery_df['balls_left']

In [ ]:
delivery_df.head()

In [ ]:
def result(row):
    return 1 if row['batting_team']==row['winner'] else 0
    

In [ ]:
delivery_df['result'] = delivery_df.apply(result, axis=1)

In [ ]:
delivery_df.sample()

In [ ]:
final=delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [ ]:
final=final.sample(final.shape[0],random_state=18)


In [ ]:
final.isnull().sum()

In [ ]:
final.drop(columns=['city'], inplace=True)



In [ ]:
final.shape

In [ ]:
final=final.dropna()

In [ ]:
final=final[final['balls_left']!=0]

In [ ]:
final.describe()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(final.iloc[:,:-1],final['result'],test_size=0.2,random_state=18)

In [ ]:
y_test

In [2]:
import sklearn
print(sklearn.__version__)


1.6.1


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
trf=ColumnTransformer([
    ['trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team']]
],remainder='passthrough')

In [181]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [189]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',RandomForestClassifier())
]
    
)
pipe1=Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
]
    
)

In [190]:
pipe1.fit(x_train,y_train)
pipe.fit(x_train,y_train)

C:\Users\Taran\anaconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[['trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team']]])),
                ('step2', RandomForestClassifier())])

In [195]:
y_pred1=pipe1.predict(x_test)
y_pred=pipe.predict(x_test)

In [196]:
from sklearn.metrics import accuracy_score

In [197]:
accuracy_score(y_pred,y_test)

0.99345581462989

In [198]:
accuracy_score(y_pred1,y_test)

0.8000872558049348

In [199]:
## accuracy of random forsest is greater but we need probabilities ..so for smooth probabilty we will use logistic regression 

In [201]:
 import pickle
pickle.dump(pipe1,open('pipe.pkl','wb'))